# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arenaコンペティションの人間による好み予測タスクに取り組んでいます。本コンペティションの目的は、大規模言語モデル（LLM）によって生成されたテキストの中から、ユーザーがどの応答を好むかを予測することです。

### 取り組んでいる問題
研究者は、ユーザーからのテキストプロンプトに対する2つのモデル（AとB）の応答を比較し、人間の好みを予測するためのモデルを開発しています。本Notebookでは、与えられたデータを処理し、LLM（具体的にはLlamaモデル）とLightGBMを組み合わせて、各モデルの勝者（どちらのモデルの応答が好まれるか）を分類するタスクを行っています。

### 使用する手法とライブラリ
1. **ライブラリのインストール**:
   - `bitsandbytes`, `transformers`, `tokenizers`, `peft`などのライブラリを使用し、LLMの効率的な推論及びパラメータの微調整に活用しています。

2. **データ加工**:
   - テストデータを読み込み、テキストを前処理してモデル入力形式に整形します。

3. **トークン化**:
   - `transformers`ライブラリを用いて、テキストをトークン化し、モデルに適した形式に変換しています。

4. **モデルのロードと設定**:
   - LlamaモデルをGPUにロードし、量子化（BitsAndBytes）を使用してメモリ効率を向上させています。
   - PEFT（Parameter-Efficient Fine-Tuning）を使用して、LoRa（Low-Rank Adaptation）に基づく設定を行い、モデルの重みをロードしています。

5. **推論の実施**:
   - 自動混合精度（autocast）を利用して、効率的に推論を行います。推論結果をデータフレームに格納します。

6. **LightGBMによるモデルの統合**:
   - テキスト特徴量を抽出し、LightGBMモデルも用いて予測を行い、最終的にLlamaモデルによる予測結果とブレンドします。

7. **結果の保存**:
   - ブレンドした予測結果をCSVファイルに保存し、コンペティションへの提出用データ形式を整えています。

このNotebookは、複数の機械学習アルゴリズムと効率的なデータ処理手法を駆使して、LLMによる応答の好み予測を体系的に解決するアプローチを示しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語の簡単な解説を列挙します。

1. **BitsAndBytes**:
    - モデルのメモリ使用量を削減し、より効率的な計算を可能にするための技術やライブラリ。特に、8ビットの浮動小数点数でモデルの重みを表現し、メモリ使用量を削減する。

2. **PEFT (Parameter-Efficient Fine-Tuning)**:
    - モデルのパラメータを効果的に微調整するためのアプローチ。少ないパラメータの変更で性能を向上させる手法で、特に大規模モデルにおいて、効率的に学習することができる。

3. **LoraConfig**:
    - PEFTの一部で、Low-Rank Adaptation (LoRA) の設定を定義するクラス。モデルの重みを低次元の行列に圧縮し、少ないリソースで微調整を行う。

4. **autocast**:
    - 自動的に混合精度計算を行うための機能。計算過程で使用する数値の精度（16ビット、32ビットなど）を動的に切り替えることで、計算の効率を上げたりメモリ使用量を削減する。

5. **アテンションマスク (Attention Mask)**:
    - モデルが入力データ中のどの部分に注目すべきかを示すためのマスク。トークンが無視されるべきか、処理されるべきかを指定するのに使用される。

6. **デバイスマップ（device_map）**:
    - 複数の計算デバイス（CPUやGPU）を使用してモデルを分散させるための設定。どの部分のモデルをどのデバイスに配置するかを指定する。

7. **シンメトリック対数変換 (Symlog transformation)**:
    - データのスケールを調整し、特に外れ値の影響を軽減させるために使用される変換方法。数値の正の値と負の値の対数を取る際に使用する。

8. **LightGBM**:
    - Microsoftが開発した、決定木に基づく勾配ブースティングフレームワーク。大規模データを効率的に扱えるように設計されており、高速かつ高精度な分類を実現する。

9. **CountVectorizer**:
    - テキストデータを数値の特徴に変換するための方法。特定の語の出現回数を数え、それを特徴量として使用する。

10. **ガーベジコレクション (Garbage Collection)**:
    - プログラム内で不要になったメモリを自動的に解放する機能。メモリリークを防ぎ、プログラムのパフォーマンスを維持する。

これらの用語の理解は、ノートブックの内容をより深く理解するために役立ちます。

---


In [ ]:
# bitsandbytesをインストールします。-qオプションは静かにインストールを行います。
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
# transformersをインストールします。-qオプションは静かにインストールを行います。
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
# tokenizersをインストールします。-qオプションは静かにインストールを行います。
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
# peftをインストールします。-qオプションは静かにインストールを行います。
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

このノートブックの作業は、以下のノートブックからインスパイアを受けています：
* https://www.kaggle.com/code/ivanvybornov/llama3-8b-lgbm-tfidf
* https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

## もしこれが役に立ちましたら、評価をいただけると嬉しいです

## ライブラリのインポート


In [ ]:
from threading import Thread
import gc
import os
import io
import json
import random
import pickle
import zipfile
import datetime
import time

# PyTorchをインポート
import torch
# NumPyをインポート
import numpy as np
# Pandasをインポート
import pandas as pd
# Transformersライブラリから必要なクラスをインポート
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
# PEFT（Parameter-Efficient Fine-Tuning）関連のクラスをインポート
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
# 自動混合精度を使用するためのクラスをインポート
from torch.cuda.amp import autocast
# ディスプレイ用の機能をインポート
from IPython.display import display
# PyTorchの関数をインポート
import torch.nn.functional as F
# tokenizersをインポート
import tokenizers

In [ ]:
# CUDAのメモリ効率的なSDPを有効化
torch.backends.cuda.enable_mem_efficient_sdp(True)
# CUDAのフラッシュSDPを有効化
torch.backends.cuda.enable_flash_sdp(True)

# モデル名の設定
MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
# 重みのパスの設定
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'
# 最大入力長の設定
MAX_LENGTH = 2048
# バッチサイズの設定
BATCH_SIZE = 4
# デバイスの設定（GPUを使用）
DEVICE = torch.device("cuda")

## データの準備 


In [ ]:
# テストデータとサンプル提出データを読み込みます
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

In [ ]:
# リスト内の文字列を連結する関数
def process(input_str):
    # 余分な括弧を取り除く
    stripped_str = input_str.strip('[]')
    # 各文を取り出してリストに格納
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    # 文を空白で連結して返す
    return  ' '.join(sentences)

# 各列に対してprocess関数を適用
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

# サンプル提出データとテストデータの最初の5行を表示
display(sample_sub)
display(test.head(5))

# モデル用のテキストを準備します
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + test['response_b']
# 最初のテキストを表示
print(test['text'][0])

## トークン化


In [ ]:
# トークナイザーをロードします
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

# テキストデータをトークン化
tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

# トークンIDとアテンションマスクをデバイスに移動
INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# テンソルをCPUに移動し、リストに変換
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

# 入力IDとアテンションマスクからデータフレームを作成
data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
data[:2]

## モデルのロード 
> 各GPUに1つのモデルをロードします。  


In [ ]:
# BitsAndBytesの設定
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットの重みをロード
    bnb_8bit_compute_dtype=torch.float16,  # 計算に使用するデータ型
    bnb_8bit_use_double_quant=False)  # ダブル量子化を使用しない

# GPU 0にベースモデルをロード
device0 = torch.device('cuda:0')

# LlamaForSequenceClassificationモデルをロード
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベル数を設定
    torch_dtype=torch.float16,  # 使用するデータ型を設定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:0')  # GPU 0にマッピング
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDの設定

In [ ]:
# GPU 1にベースモデルをロード
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベル数を設定
    torch_dtype=torch.float16,  # 使用するデータ型を設定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:1')  # GPU 1にマッピング
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDの設定

## 重みのロード 



In [ ]:
# LoRaの設定
peft_config = LoraConfig(
    r=16,  # ローラ次元
    lora_alpha=32,  # ローラアルファ
    lora_dropout=0.10,  # ドロップアウト率
    bias='none',  # バイアス設定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクのタイプ
    target_modules=['o_proj', 'v_proj'])  # 対象モジュールの設定

In [ ]:
# PEFTモデルを取得
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
# 重みをロード
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_0.eval()  # 評価モードに設定

# モデル1を設定
model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータを表示
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

In [ ]:
gc.collect()  # ガーベジコレクションを実行

## 推論


In [ ]:
# 推論を行う関数
def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    generated_class_a = []  # 出力クラスA
    generated_class_b = []  # 出力クラスB
    generated_class_c = []  # 出力クラスC

    model.eval()  # 評価モードに設定
    
    # データフレームをバッチサイズごとに繰り返す
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに移動
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチのアテンションマスクをデバイスに移動
        
        with torch.no_grad():  # 勾配計算を無効にする
            with autocast():  # 自動混合精度を使用
                outputs = model(
                    input_ids=batch_input_ids,  # 入力IDをモデルに渡す
                    attention_mask=batch_attention_mask  # アテンションマスクをモデルに渡す
                )
        
        # 出力の確率を計算
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        # 各クラスの確率を格納
        generated_class_a.extend(probabilities[:, 0])
        generated_class_b.extend(probabilities[:, 1])
        generated_class_c.extend(probabilities[:, 2])
    
    # データフレームに結果を追加
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  # GPUメモリをクリア

    return df

In [ ]:
st = time.time()  # 処理開始時間を記録

N_SAMPLES = len(data)  # サンプル数を取得

# データを2つのサブセットに分割
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()  # 前半のデータ
sub2 = data.iloc[half:N_SAMPLES].copy()  # 後半のデータ

# スレッド内で推論を実行する関数
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 推論結果を格納

# スレッドからの結果を格納するための辞書
results = {}

In [ ]:
# スレッドの開始
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # スレッド0
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # スレッド1

t0.start()  # スレッド0を開始
t1.start()  # スレッド1を開始

# すべてのスレッドが終了するのを待つ
t0.join()
t1.join()

# 元のデータフレームに結果を結合
data = pd.concat([results[0], results[1]], axis=0)

print(f"処理が完了しました。総時間: {time.time() - st}")  # 処理時間を表示

# サンプル提出に結果を追加
TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = data[TARGETS]

In [ ]:
llama_preds = data[TARGETS].values  # 予測結果を配列に格納

## LGBM + tfidf


In [ ]:
TAG = 'lmsys-chatbot-arena'  # コンペティションのタグ
RUNPOD = os.path.exists('/workspace/')  # 実行環境の確認
KAGGLE = not RUNPOD  # Kaggle環境であるかの確認
if KAGGLE: 
    print('kaggle')  # Kaggle環境である場合、メッセージを表示

In [ ]:
try:
    import pandas as pd  # Pandasをインポート
except:
    # Kaggle環境でPandasがない場合、インストール
    !pip install -q kaggle
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm 
    !pip install -q protobuf 
    !pip install -q numba

In [ ]:
# データのパスを設定
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

# 実行環境がRUNPODの場合
if RUNPOD:
    # Kaggle APIの設定ファイルが存在しない場合
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle  # ディレクトリを作成
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # 設定ファイルをコピー
        !chmod 600 /root/.kaggle/kaggle.json  # アクセス権を設定

    # データファイルが存在しない場合、Kaggleからダウンロード
    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/ 
        
    # ダウンロード後、データを展開
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')

In [ ]:
# パスを設定
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
print(MODEL_PATH)  # モデルパスを表示

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'  # コードパスの設定
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # 保存パスの設定

In [ ]:
# トークナイザーの並列化を無効にする
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
# トレーニングデータ、テストデータ、サンプル提出データを読み込む
train = pd.read_csv(open(DATA + 'train.csv', 'r'))
test = pd.read_csv(open(DATA + 'test.csv', 'r'))
sample = pd.read_csv(DATA + 'sample_submission.csv')
# データの長さを表示
print(len(train), len(test))

In [ ]:
params = {}
if False: 
    pass;
    params['subsample'] = 30
else:
    params['fold'] = -1  # フォールドを設定


params['n_epochs'] = 1  # エポック数の設定
params['n_lgb'] = 1  # LightGBMの数の設定
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデルを設定

In [ ]:
# params = {}
FULL = params.get('fold', 0) < 0  # フルデータかどうかを設定
N_FOLDS = int(params.get('n_folds', 3));  # フォールド数の設定
FOLD = int(params.get('fold', 0))  # 現在のフォールドの設定
SEED = int(params.get('seed', 3))  # シード値の設定
SS = int(params.get('subsample', 1))  # サブサンプル数の設定

print(N_FOLDS, FOLD, SEED, SS)  # 設定値を表示

In [ ]:
from sklearn.model_selection import StratifiedKFold

# StratifiedKFoldの設定を行う関数
def get_folds(train): 
    # StratifiedKFoldを使ってフォールドを作成
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))

# トレーニングとテスト用のIDを取得
train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
# サブサンプル数に基づいてIDを選択
if SS > 1:
    train_ids, test_ids = train_ids[::SS], test_ids[::SS]

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set()  # IDの重複がないことを確認

In [ ]:
# 現在のマイクロ秒に基づいてシードを設定します
torch.manual_seed(datetime.datetime.now().microsecond)
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)

In [ ]:
# トレーニング、推論、および保存のフラグを設定
TRAIN = False
INFER = True 
SAVE = False

In [ ]:
# LightGBMと特徴量抽出のためのライブラリをインポート
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# LightGBM関連のフラグ
LGB = True
# トレーニングするかどうかのフラグ
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0
# 推論を行うかどうかのフラグ
INFER_LGB = not TRAIN and LGB

In [ ]:
# 事前に保存したCountVectorizerモデルを読み込む
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # 主要なCountVectorizer
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # カスタムCountVectorizer

In [ ]:
# シンメトリック対数変換を行う関数
def symlog(x):
    return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)

# Dense行列を処理する関数
def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # dense形式に変換
    x = symlog(x)  # シンメトリック対数変換を適用
    return x

# 特徴量を取得する関数
def get_features(df):
    # promptから特徴量を抽出
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    # response_aから特徴量を抽出
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    # response_bから特徴量を抽出
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    # 特徴量を計算
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat), 
        ])
    try: 
        # 投票モデルの数で特徴量を割る
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except:
        pass

    extras = []  # 追加の特徴量を格納するリスト
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # 特徴量として使用する文字列
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 特定の文字のカウントを追加
            
    # 文字列の長さと単語数を追加
    extras.append(df[c].str.len())
    extras.append(df[c].str.split().apply(lambda x: len(x)))
    
    extras = np.stack(extras, axis = 1)  # スタックして配列に変換
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 特徴量を拡張
    return np.hstack([v, extras])  # 全特徴量を結合して返す

In [ ]:
# 事前に保存したLightGBMモデルを読み込む
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))

In [ ]:
# 推論を行う場合の処理
if INFER and params.get('n_lgb', 1) > 0:
    df = test  # テストデータセットを使用
    yps = []  # 予測値を格納するリスト
    b = 1000  # バッチサイズ
    # テストデータをバッチごとに処理
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])  # 特徴量を取得
        ypms = []  # 各モデルの予測を格納するリスト
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))  # 各モデルで予測
        
        yps.append(np.stack(ypms).mean(0))  # 予測値の平均を格納
        print('.', end = '')  # 進行状況を表示
        
        # メモリ管理
        if len(yps) % 2 == 0:
            gc.collect()  # ガーベジコレクションを実行
    print()

    yp = np.concatenate(yps)  # すべての予測を結合

In [ ]:
lgb_preds = yp  # LightGBMの予測値を格納

## 予測のブレンド

$\operatorname{preds} = 0.05 \cdot \operatorname{lgbm \ boosting \ preds} + 0.8 \cdot \operatorname{llama \ preds}$



In [ ]:
# 予測のブレンドを行う
lgb_wt = 0.05  # LightGBMの重み
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 予測の加重平均

In [ ]:
# 結果をデータフレームに保存
out = pd.DataFrame(preds, index=df.id, columns=train.columns[-3:])  # 予測結果のデータフレームを作成
display(out.head())  # 最初の数行を表示

In [ ]:
# 結果をCSVファイルに保存
out.to_csv('submission.csv')  # 提出用ファイルを保存